<a href="https://colab.research.google.com/github/athira-tech1/C-_-/blob/main/EXistingFramework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install cryptography numpy opencv-python

In [7]:
import os
import time
import cv2
import numpy as np
import hashlib
from cryptography.hazmat.primitives.asymmetric import x25519
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.backends import default_backend

def generate_key_pair():
    private_key = x25519.X25519PrivateKey.generate()
    public_key = private_key.public_key()
    return private_key, public_key

def compute_hmac(data, key):
    h = hashlib.blake2b(digest_size=32)  # 32 bytes = 256 bits
    h.update(data)
    h.update(key)
    return h.hexdigest()

def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    nodes = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert frame to bytes
        data = frame.tobytes()
        nodes.append(data)

    cap.release()
    return nodes

def compute_video_hash(video_path):
    nodes = process_video(video_path)
    key = os.urandom(32)  # Random key for HMAC
    hmac_values = []

    for data in nodes:
        hmac_value = compute_hmac(data, key)
        hmac_values.append(hmac_value)

    return hmac_values, key

def verify_video_integrity(original_hmacs, video_path, key):
    nodes = process_video(video_path)
    hmac_values = []

    for data in nodes:
        hmac_value = compute_hmac(data, key)
        hmac_values.append(hmac_value)

    return hmac_values

def main(original_video_path, video_to_check_path):
    # Start timing the processing
    start_time = time.time()

    # Compute the original video hash
    original_hmacs, key = compute_video_hash(original_video_path)

    # End timing the processing
    processing_time = time.time() - start_time

    # Start timing the verification
    start_time = time.time()

    # Verify the integrity of the video to check
    checked_hmacs = verify_video_integrity(original_hmacs, video_to_check_path, key)

    # End timing the verification
    verification_time = time.time() - start_time

    # Compare the original and checked HMACs
    altered = original_hmacs != checked_hmacs

    # Get the file types
    original_file_type = os.path.splitext(original_video_path)[1]
    checked_file_type = os.path.splitext(video_to_check_path)[1]

    # Output results
    print(f"Original Video HMACs: {original_hmacs}")
    print(f"Checked Video HMACs: {checked_hmacs}")
    print(f"File Type of Original Video: {original_file_type}")
    print(f"File Type of Video to Check: {checked_file_type}")
    print(f"Processing Time (seconds): {processing_time:.3f}")
    print(f"Verification Time (seconds): {verification_time:.3f}")
    print(f"Video Altered: {'Yes' if altered else 'No'}")

# Example usage
original_video_path = '/content/drive/MyDrive/Testdata/Original1.mp4'  # Replace with your original video file
video_to_check_path = '/content/drive/MyDrive/Testdata/framerate.mp4'  # Replace with the video you want to check

main(original_video_path, video_to_check_path)

Original Video HMACs: ['...']
Checked Video HMACs: ['...']
File Type of Original Video: .mp4
File Type of Video to Check: .mp4
Processing Time (seconds): 13.591
Verification Time (seconds): 14.367
Video Altered: Yes
